In [66]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [67]:
startdate =  datetime(2007, 1, 3)
testdate_start = datetime(2020, 12, 15)
enddate =  datetime(2023, 12, 15)

GIST_NAME = 'thesis_1'

# tickerStrings = ['XLY', 'EEM', 'DBC', 'XLE', 'XLF', 'GSG']
tickerStrings = ['VWO', 'SPY', 'VNQ', 'LQD', 'DBC']

In [68]:
variable_name = "ZIPLINE_ROOT"
zipline_root = '.'# os.environ[variable_name]

In [69]:
df_list = list()
for ticker in tickerStrings:
    data = yf.download(ticker, group_by="Ticker", start=startdate, end=enddate)
    data['ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
    df_list.append(data)

# combine all dataframes into a single dataframe
df = pd.concat(df_list)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [70]:
df[df['ticker'] == 'GSG']

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,


In [71]:
rows_with_nan = df[df.isna().any(axis=1)]


In [72]:
df.reset_index(inplace=True)


In [73]:
df.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'ticker']
df = df[['ticker','date','open','close','low','high','volume']]

## Data for training/Validation

In [74]:
def data_to_returns(X):
    rets = np.vstack([
        X[X['ticker'] == t]['close'].pct_change()[1:] for t in tickerStrings
    ]).T
    
    return pd.DataFrame(rets)


In [78]:
train_data = df[df['date'] < testdate_start]
data_to_returns(train_data).to_csv('train_data_0.csv')

In [80]:
train_data_U = df[df['date'] < testdate_start + relativedelta(years=1)] # wil be used by a model that will select weights after a year of backtest
data_to_returns(train_data_U).to_csv('train_data_1.csv')

## Data for ingesting

In [77]:
ingesting_data = df#df[df['date'] >= testdate_start] # All the data is ingested

In [12]:
for t in tickerStrings:
    ticker = yf.Ticker(t)

    # Specify the start and end dates for the period you're interested in

    # Fetch the historical data for the specified period
    historical_data = ticker.history(start=startdate, end=enddate)
    ingesting_data.loc[ingesting_data['ticker'] == t, 'dividend'] = historical_data['Dividends'].values
    ingesting_data.loc[ingesting_data['ticker'] == t, 'split'] = 1 #historical_data['Stock Splits'].values


In [13]:
# ['XLY', 'EEM', 'DBC', 'XLE', 'XLF', 'GSG']

In [14]:
tmp = ingesting_data[ingesting_data['ticker'] == 'VWO']
tmp[tmp['split'] != 0]

,ticker,date,open,close,low,high,volume,dividend,split
0,VWO,2007-01-03,39.134998,38.950001,38.700001,39.549999,1176000,0.0,1.0
1,VWO,2007-01-04,38.700001,38.525002,38.270000,38.724998,1115600,0.0,1.0
2,VWO,2007-01-05,38.014999,37.450001,37.325001,38.125000,1145800,0.0,1.0
3,VWO,2007-01-08,37.660000,37.860001,37.439999,37.884998,496200,0.0,1.0
4,VWO,2007-01-09,37.650002,36.955002,36.615002,37.650002,934600,0.0,1.0
...,...,...,...,...,...,...,...,...,...
4263,VWO,2023-12-08,40.130001,40.209999,40.049999,40.310001,7658200,0.0,1.0
4264,VWO,2023-12-11,40.189999,40.389999,40.139999,40.410000,13378300,0.0,1.0
4265,VWO,2023-12-12,40.259998,40.389999,40.099998,40.410000,10349000,0.0,1.0
4266,VWO,2023-12-13,40.200001,40.770000,40.029999,40.770000,11221200,0.0,1.0


In [15]:
ingesting_data[ingesting_data['volume'] == 0]

,ticker,date,open,close,low,high,volume,dividend,split


In [16]:
df

,ticker,date,open,close,low,high,volume,dividend,split
0,VWO,2007-01-03,39.134998,38.950001,38.700001,39.549999,1176000,0.0,1.0
1,VWO,2007-01-04,38.700001,38.525002,38.270000,38.724998,1115600,0.0,1.0
2,VWO,2007-01-05,38.014999,37.450001,37.325001,38.125000,1145800,0.0,1.0
3,VWO,2007-01-08,37.660000,37.860001,37.439999,37.884998,496200,0.0,1.0
4,VWO,2007-01-09,37.650002,36.955002,36.615002,37.650002,934600,0.0,1.0
...,...,...,...,...,...,...,...,...,...
21335,DBC,2023-12-08,23.040001,23.000000,22.930000,23.120001,1366100,0.0,1.0
21336,DBC,2023-12-11,22.860001,22.920000,22.760000,22.940001,3260700,0.0,1.0
21337,DBC,2023-12-12,22.670000,22.570000,22.540001,22.709999,1983200,0.0,1.0
21338,DBC,2023-12-13,22.600000,22.830000,22.570000,22.830000,1613700,0.0,1.0


In [17]:
#tmp_df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
# ingesting_data['dividend'] = 0.0
# ingesting_data['split'] = 1.0


import os


DIR_NAME = os.path.join(zipline_root, GIST_NAME, 'daily')
if not os.path.exists(DIR_NAME):
    os.makedirs(DIR_NAME)

In [18]:
DIR_NAME

'.\\thesis_1\\daily'

In [19]:
for t in tickerStrings:
    file_path = os.path.join(DIR_NAME, f'{t}.csv')
    tmp_df = ingesting_data[ingesting_data['ticker']==t][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
    
    # fix data problems
    tmp_row_1 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2022-06-21'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
    tmp_row_1['date'] = datetime(2022, 6, 20)
    
    tmp_row_2 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2023-06-20'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
    tmp_row_2['date'] = datetime(2023, 6, 19)
    
    
    tmp_df = tmp_df.append(tmp_row_1, ignore_index=True)
    tmp_df = tmp_df.append(tmp_row_2, ignore_index=True)
    
    tmp_df = tmp_df.sort_values(by='date')
    
    print(tmp_df[tmp_df.isna().any(axis=1)])
    
    tmp_df.to_csv(file_path, index=False)

C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_1 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2022-06-21'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_2 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2023-06-20'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_1 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2022-06-21'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_46

Empty DataFrame
Columns: [date, open, high, low, close, volume, dividend, split]
Index: []
Empty DataFrame
Columns: [date, open, high, low, close, volume, dividend, split]
Index: []
Empty DataFrame
Columns: [date, open, high, low, close, volume, dividend, split]
Index: []
Empty DataFrame
Columns: [date, open, high, low, close, volume, dividend, split]
Index: []
Empty DataFrame
Columns: [date, open, high, low, close, volume, dividend, split]
Index: []


C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_1 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2022-06-21'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_2 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2023-06-20'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_4696\138923210.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp_row_1 = ingesting_data[ingesting_data['ticker'] == t][ingesting_data['date'] == '2022-06-21'][['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']]
C:\Users\r0913246\AppData\Local\Temp\ipykernel_46

In [129]:
df[df['dividend'] != 0]

,ticker,date,open,close,low,high,volume,dividend,split
246,VWO,2007-12-24,52.724998,53.224998,52.630001,53.240002,701600,0.9845,0.0
499,VWO,2008-12-24,23.090000,22.990000,22.610001,23.139999,2772500,1.1780,0.0
751,VWO,2009-12-24,40.619999,40.639999,40.509998,40.709999,2458300,0.5450,0.0
1001,VWO,2010-12-22,46.880001,47.049999,46.840000,47.099998,11354400,0.8150,0.0
1253,VWO,2011-12-21,37.919998,38.180000,37.570000,38.209999,23553400,0.9060,0.0
...,...,...,...,...,...,...,...,...,...
17313,DBC,2007-12-17,31.549999,30.520000,30.240000,31.549999,294000,0.7600,0.0
17564,DBC,2008-12-15,21.780001,20.950001,20.670000,21.950001,1382400,0.3400,0.0
20087,DBC,2018-12-24,14.510000,14.390000,14.390000,14.560000,1525700,0.1890,0.0
20338,DBC,2019-12-23,15.820000,15.860000,15.790000,15.870000,837500,0.2540,0.0


In [130]:
df

,ticker,date,open,close,low,high,volume,dividend,split
0,VWO,2007-01-03,39.134998,38.950001,38.700001,39.549999,1176000,0.0,0.0
1,VWO,2007-01-04,38.700001,38.525002,38.270000,38.724998,1115600,0.0,0.0
2,VWO,2007-01-05,38.014999,37.450001,37.325001,38.125000,1145800,0.0,0.0
3,VWO,2007-01-08,37.660000,37.860001,37.439999,37.884998,496200,0.0,0.0
4,VWO,2007-01-09,37.650002,36.955002,36.615002,37.650002,934600,0.0,0.0
...,...,...,...,...,...,...,...,...,...
21335,DBC,2023-12-08,23.040001,23.000000,22.930000,23.120001,1366100,0.0,0.0
21336,DBC,2023-12-11,22.860001,22.920000,22.760000,22.940001,3260700,0.0,0.0
21337,DBC,2023-12-12,22.670000,22.570000,22.540001,22.709999,1983200,0.0,0.0
21338,DBC,2023-12-13,22.600000,22.830000,22.570000,22.830000,1613700,0.0,0.0
